In [1]:
import numpy as np
import pandas as pd
import dowhy
import matplotlib.pyplot as plt
from causalgraphicalmodels import CausalGraphicalModel
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',35)
import sys
sys.path.append("../scripts/")
#from plot import plot_distribution
import warnings
warnings.filterwarnings('ignore')
from dowhy import CausalModel
from IPython.display import Image, display

In [2]:
df = pd.read_csv("../data/processed_data.csv")
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


* Feature selection - logistic regression

In [3]:
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split

X = df.drop(['diagnosis'],axis=1)
y = df.diagnosis.values

print('Train: ', X.shape)
print('Hold-out: ', y.shape)

Train:  (569, 30)
Hold-out:  (569,)


In [4]:
log = LogisticRegression()
log.fit(X,y)
feature_importance = log.coef_[0]
#for i,v in enumerate(feature_importance):
   # print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#plt.bar([x for x in range(len(feature_importance))], feature_importance)

In [5]:
features = pd.DataFrame()
features['attributes'] = X.columns.to_list()
features['score'] = log.coef_[0]
features.head()

,attributes,score
0,radius_mean,-0.949807
1,texture_mean,-0.458173
2,perimeter_mean,-0.275798
3,area_mean,0.016071
4,smoothness_mean,0.035370


In [6]:
causal_graph = """
digraph{
radius_mean;
texture_mean;
perimeter_mean;
area_mean;
smoothness_mean;
concavity_mean;
concave_points_mean;
radius_se;
perimeter_se;
area_se;
smoothness_se;
concave_points_se;
radius_worst;
texture_worst;
perimeter_worst;
area_worst;
smoothness_worst;
concavity_worst;
concave_points_worst;
symmetry_worst;
fractal_dimension_worst;
U[label="Unobserved Confounders"];
radius_mean -> perimeter_mean;radius_mean -> area_mean;radius_mean -> smoothness_mean;radius_mean->high_radius;
concave_points_mean->concavity_mean;radius_se -> perimeter_se;radius_se -> area_se;radius_se -> smoothness_se;
radius_worst -> perimeter_worst;radius_worst -> area_worst;radius_worst -> smoothness_worst;concave_points_worst->concavity_worst;
radius_se -> perimeter_se;radius_se -> area_se;radius_se -> smoothness_se;radius_worst -> perimeter_worst;
radius_worst -> area_worst;radius_worst -> smoothness_worst;concave_points_worst->concavity_worst;

U->perimeter_mean;U->area_mean;U->smoothness_mean;U->texture_mean;U->concavity_mean;U->perimeter_se;U->area_se;U->smoothness_se;
U->smoothness_se;U->concave_points_se;U->perimeter_worst;U->area_worst;U->smoothness_worst;U->texture_worst;U->concavity_worst;
U->symmetry_worst;U->fractal_dimension_worst;U->high_radius;U->diagnosis;

texture_mean->diagnosis;perimeter_mean->diagnosis;area_mean->diagnosis;smoothness_mean->diagnosis;concavity_mean->diagnosis;
perimeter_se->diagnosis;area_se->diagnosis;smoothness_se->diagnosis;concave_points_se->diagnosis;high_radius->diagnosis;
texture_worst->diagnosis;perimeter_worst->diagnosis;area_worst->diagnosis;smoothness_worst->diagnosis;
concavity_worst->diagnosis;concave_points_worst->diagnosis;symmetry_worst->diagnosis;fractal_dimension_worst->diagnosis;
}
"""

In [7]:
#area and perimeter functions of radius
from causalgraphicalmodels import CausalGraphicalModel
causal = CausalGraphicalModel(
    nodes=['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'concavity_mean', 'concave_points_mean', 'radius_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'concave points_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis',
       'high_radius','U'],
    edges=[
        ("radius_mean", "perimeter_mean"), 
        ("radius_mean", "area_mean"),
        ("radius_mean","smoothness_mean"),
        ("concave_points_mean","concavity_mean"),
        ("radius_mean","high_radius"),
        ("radius_se", "perimeter_se"), 
        ("radius_se", "area_se"),
        ("radius_se","smoothness_se"),
        ("radius_worst", "perimeter_worst"), 
        ("radius_worst", "area_worst"),
        ("radius_worst","smoothness_worst"),
        ("concave_points_worst","concavity worst"),
        ("U","perimeter_mean"),
        ("U","area_mean"),
        ("U","texture_mean"),
        ("U","smoothness_mean"),
        ("U","concavity_mean"),
        ("U","perimeter_se"),
        ("U","area_se"),
        ("U","concave points_se"),
        ("U","smoothness_se"),
        ("U","perimeter_worst"),
        ("U","area_worst"),
        ("U","texture_worst"),
        ("U","smoothness_worst"),
        ("U","concavity_worst"),
        ("U","symmetry_worst"),
        ("U","fractal_dimension_worst"),
        ("U","high_radius"),
        ("U","diagnosis"),
        ("high_radius","diagnosis"),
        ("perimeter_mean","diagnosis"),
        ("area_mean","diagnosis"),
        ("texture_mean","diagnosis"),
        ("smoothness_mean","diagnosis"),
        ("concavity_mean","diagnosis"),
        ("smoothness_mean","diagnosis"),
        ("concavity_mean","diagnosis"),
        ("perimeter_se","diagnosis"),
        ("area_se","diagnosis"),
        ("smoothness_se","diagnosis"),
        ("concavity_se","diagnosis"),
        ("smoothness_se","diagnosis"),
        ("concave points_se","diagnosis"),
        ("concave points_worst","diagnosis"),
        ("perimeter_worst","diagnosis"),
        ("area_worst","diagnosis"),
        ("texture_worst","diagnosis"),
        ("smoothness_worst","diagnosis"),
        ("concavity_worst","diagnosis"),
        ("smoothness_worst","diagnosis"),
        ("symmetry_worst","diagnosis"),
        ("fractal_dimension_worst","diagnosis")
    ]
)
# draw return a graphviz `dot` object, which jupyter can render
causal.draw()

ExecutableNotFound: failed to execute 'dot', make sure the Graphviz executables are on your systems' PATH

In [8]:
#pip install graphviz